# Install and Import Libraries

In [1]:
# Install necessary packages
!pip install --upgrade tensorflow tensorflow-datasets

In [2]:
# Import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load and Preprocess the Dataset

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Enable memory growth for GPUs (avoid crashes)
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Load PatchCamelyon dataset efficiently
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    'patch_camelyon',
    split=['train[:80%]', 'train[80%:]', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Verify loaded dataset
print(f"Dataset Info:\n{ds_info}")

# Example: view a sample image
import matplotlib.pyplot as plt

def show_sample(image, label):
    plt.imshow(image.numpy())
    plt.title(f"Label: {label.numpy()}")
    plt.axis('off')

for image, label in ds_train.take(1):
    show_sample(image, label)


In [4]:
# Define preprocessing and augmentation functions
def preprocess(image, label):
    # Normalise images to [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize images to ensure consistent input size (if required by your models)
    image = tf.image.resize(image, [96, 96])
    return image, label

# Augmentation function according to PDM plan (rotation, flip, contrast)
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.clip_by_value(image, 0.0, 1.0)  # Ensure pixel values are within [0,1]
    return image, label

# Apply preprocessing, augmentation, batching, caching, and prefetching
batch_size = 64

# Training dataset pipeline
ds_train = (
    ds_train
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(1000)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Test dataset pipeline (no augmentation, just preprocessing)
ds_test = (
    ds_test
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Check a batch of training images
for images, labels in ds_train.take(1):
    plt.figure(figsize=(15, 5))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i].numpy())
        plt.title(f'Label: {labels[i].numpy()}')
        plt.axis('off')
    plt.show()


# Define the Custom MM-SEN-Inspired Model

In [5]:
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, Concatenate, Multiply, Reshape
from tensorflow.keras.models import Model

def simple_attention_block(x):
    channels = x.shape[-1]
    attention = GlobalAveragePooling2D()(x)
    attention = Dense(channels // 2, activation='relu')(attention)
    attention = Dense(channels, activation='sigmoid')(attention)
    attention = Reshape((1, 1, channels))(attention)
    return Multiply()([x, attention])

input_layer = Input(shape=(96, 96, 3))

branch1 = Conv2D(32, (3,3), activation='relu', padding='same')(input_layer)
branch2 = Conv2D(32, (5,5), activation='relu', padding='same')(input_layer)
branch3 = Conv2D(32, (7,7), activation='relu', padding='same')(input_layer)

combined = Concatenate()([branch1, branch2, branch3])
attention_output = simple_attention_block(combined)
x = GlobalAveragePooling2D()(attention_output)
output_layer = Dense(1, activation='sigmoid')(x)

custom_mmsen_model = Model(inputs=input_layer, outputs=output_layer)
custom_mmsen_model.summary()


# Compile and Train the Model (40 epochs)

In [6]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set random seed for reproducibility
tf.random.set_seed(42)

# Compile the custom MM-SEN model
custom_mmsen_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Add EarlyStopping to stop training if no improvement in validation loss for 10 epochs
early_stopping = EarlyStopping(
    monitor='val_loss',          # Monitors validation loss
    patience=10,                  # Stops if no improvement for 10 epochs
    restore_best_weights=True,   # Restores model weights from the best epoch
    verbose=1
)

# Add ModelCheckpoint to save the best model during training
checkpoint = ModelCheckpoint(
    filepath='/content/custom_mmsen_best_model.h5',   # Path to save the model
    monitor='val_loss',                              # Monitors validation loss
    save_best_only=True,                              # Saves only the best model
    verbose=1
)

# Train the model with early stopping and model checkpoint
history_custom_mmsen = custom_mmsen_model.fit(
    ds_train,
    epochs=40,
    validation_data=ds_test,
    callbacks=[early_stopping, checkpoint]   # Adding both callbacks
)

# Save the final model after training
custom_mmsen_model.save('/content/custom_mmsen_final_model.h5')
print("Model saved after training!")


In [7]:
best_model = tf.keras.models.load_model('/content/custom_mmsen_best_model.h5')

# Visualisation and Evaluation After Initial Training

In [8]:
# Plotting function
import matplotlib.pyplot as plt
import numpy as np

def plot_training_curves(history):
    epochs = np.arange(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(epochs)
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['loss'], label='Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(epochs)
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['auc'], label='AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.title('AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.xticks(epochs)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Evaluation function
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        preds = model.predict(images)
        preds = np.round(preds).astype(int).flatten()
        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    print("AUC-ROC:", roc_auc_score(y_true, y_pred))

# Run evaluation and visualisation
plot_training_curves(history_custom_mmsen)
evaluate_model(custom_mmsen_model, ds_test)

In [27]:
# Plotting function
import matplotlib.pyplot as plt
import numpy as np

def plot_training_curves(history):
    epochs = np.arange(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(np.arange(1, len(epochs) + 1, step=5))  # Setting step to 5 for clear spacing
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['loss'], label='Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(np.arange(1, len(epochs) + 1, step=5))  # Setting step to 5 for clear spacing
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['auc'], label='AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.title('AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.xticks(np.arange(1, len(epochs) + 1, step=5))  # Setting step to 5 for clear spacing
    plt.legend()

    plt.tight_layout()
    plt.show()

# Evaluation function
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        preds = model.predict(images)
        preds = np.round(preds).astype(int).flatten()
        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    print("AUC-ROC:", roc_auc_score(y_true, y_pred))

# Run evaluation and visualisation
plot_training_curves(history_custom_mmsen)
evaluate_model(custom_mmsen_model, ds_test)


In [9]:
# Function to visualise predictions on 20 test images clearly
def visualize_predictions(model, dataset, num_images=20):
    plt.figure(figsize=(20, 10))
    images_shown = 0
    for images, labels in dataset:
        predictions = model.predict(images)
        preds_rounded = np.round(predictions).astype(int).flatten()

        for i in range(images.shape[0]):
            plt.subplot(4, 5, images_shown + 1)
            plt.imshow(images[i].numpy())
            plt.title(f"True: {labels.numpy()[i]} | Pred: {preds_rounded[i]}")
            plt.axis('off')

            images_shown += 1
            if images_shown >= num_images:
                plt.tight_layout()
                plt.show()
                return

# Execute clearly after your training and evaluation
visualize_predictions(custom_mmsen_model, ds_test, num_images=20)


 # Fine-tuning (Further 10 epochs)

In [10]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set random seed for reproducibility
tf.random.set_seed(42)

# Recompile the model with a lower learning rate for fine-tuning
custom_mmsen_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Early stopping to prevent overfitting during fine-tuning
early_stopping_finetune = EarlyStopping(
    monitor='val_loss',          # Monitor validation loss
    patience=3,                  # Stops if no improvement for 3 epochs
    restore_best_weights=True,   # Restores best model weights
    verbose=1
)

# Model checkpoint to save the best fine-tuned model
checkpoint_finetune = ModelCheckpoint(
    filepath='/content/custom_mmsen_best_finetune_model.h5',  # Save path
    monitor='val_loss',                                      # Monitor validation loss
    save_best_only=True,                                      # Save only the best model
    verbose=1
)

# Further fine-tuning for 10 epochs
history_finetune_custom_mmsen = custom_mmsen_model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[early_stopping_finetune, checkpoint_finetune]  # Adding callbacks
)

# Save the final fine-tuned model after training
custom_mmsen_model.save('/content/custom_mmsen_finetuned_final.h5')
print("Fine-tuned model saved after further training!")


# Final Evaluation and Visualisation After Fine-tuning

In [11]:
# Evaluate and visualise after fine-tuning (additional 10 epochs)
plot_training_curves(history_finetune_custom_mmsen)
evaluate_model(custom_mmsen_model, ds_test)

# Clearly visualise predictions on 20 test images after fine-tuning
def visualize_predictions(model, dataset, num_images=20):
    plt.figure(figsize=(20, 10))
    images_shown = 0
    for images, labels in dataset:
        predictions = model.predict(images)
        preds_rounded = np.round(predictions).astype(int).flatten()
        for i in range(images.shape[0]):
            plt.subplot(4, 5, images_shown + 1)
            plt.imshow(images[i].numpy())
            plt.title(f"True: {labels.numpy()[i]} | Pred: {preds_rounded[i]}")
            plt.axis('off')
            images_shown += 1
            if images_shown >= num_images:
                plt.tight_layout()
                plt.show()
                return

# Execute visualisation after fine-tuning
visualize_predictions(custom_mmsen_model, ds_test)


In [34]:
# Plotting function with improved y-axis scaling
import matplotlib.pyplot as plt
import numpy as np

def plot_training_curves(history):
    epochs = np.arange(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(18, 5))

    # Accuracy Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim(0.75, 0.90)  # Set y-axis range to better reflect the accuracy spread
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.legend()

    # Loss Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['loss'], label='Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.ylim(0.30, 0.45)  # Set y-axis range to better capture the loss variations
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.legend()

    # AUC-ROC Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['auc'], label='AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.title('AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.ylim(0.10, 0.99)  # Adjust y-axis range to focus on AUC differences
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.legend()

    plt.tight_layout()
    plt.show()

# Evaluation function
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        preds = model.predict(images)
        preds = np.round(preds).astype(int).flatten()
        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    print("AUC-ROC:", roc_auc_score(y_true, y_pred))

# Run evaluation and visualisation
plot_training_curves(history_finetune_custom_mmsen)
evaluate_model(custom_mmsen_model, ds_test)


In [12]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        preds = model.predict(images)
        preds = np.round(preds).astype(int).flatten()
        y_true.extend(labels.numpy())
        y_pred.extend(preds)

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Non-Metastatic', 'Metastatic'])
    disp.plot(cmap='Blues')
    plt.title('Confusion Matrix')
    plt.show()

# Run after training
plot_confusion_matrix(custom_mmsen_model, ds_test)


In [13]:
from sklearn.metrics import roc_curve, roc_auc_score

def plot_roc_curve(model, dataset):
    y_true, y_scores = [], []
    for images, labels in dataset:
        preds = model.predict(images).flatten()
        y_scores.extend(preds)
        y_true.extend(labels.numpy())

    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    auc_score = roc_auc_score(y_true, y_scores)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC curve (AUC = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.grid()
    plt.show()

# Run after training
plot_roc_curve(custom_mmsen_model, ds_test)


In [14]:
finetuned_model = tf.keras.models.load_model('/content/custom_mmsen_best_finetune_model.h5')

In [15]:
from sklearn.metrics import precision_recall_curve, average_precision_score

def plot_precision_recall_curve(model, dataset):
    y_true, y_scores = [], []
    for images, labels in dataset:
        preds = model.predict(images).flatten()
        y_scores.extend(preds)
        y_true.extend(labels.numpy())

    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    ap_score = average_precision_score(y_true, y_scores)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, label=f'PR Curve (AP = {ap_score:.4f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.grid()
    plt.show()

# Run after training
plot_precision_recall_curve(custom_mmsen_model, ds_test)


In [17]:
!pip install opencv-python

In [18]:
import tensorflow as tf
import numpy as np
import cv2

def generate_grad_cam(model, image, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([image]))
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs.numpy()[0]
    pooled_grads = pooled_grads.numpy()

    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(conv_outputs)

    heatmap = cv2.resize(heatmap, (96, 96))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap * 0.4 + (image * 255)
    superimposed_img = np.clip(superimposed_img / superimposed_img.max(), 0, 1)

    return superimposed_img


In [19]:
import matplotlib.pyplot as plt

# Example usage (clearly run with one test image)
for images, labels in ds_test.take(1):
    image = images[0].numpy()
    grad_cam_result = generate_grad_cam(custom_mmsen_model, image, last_conv_layer_name='conv2d_2')

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(grad_cam_result)
    plt.title('Grad-CAM')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [20]:
custom_mmsen_model.summary()

In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load small demo subset (20 images) from the test set
demo_dataset = ds_test.unbatch().take(20).batch(1)

In [22]:
import numpy as np

predictions = []
true_labels = []

for img, label in demo_dataset:
    pred = custom_mmsen_model.predict(img)
    predictions.append(np.round(pred).astype(int).flatten()[0])
    true_labels.append(label.numpy().flatten()[0])

In [23]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
for i, (img_batch, pred, true_label) in enumerate(zip(demo_dataset, predictions, true_labels)):
    image = img_batch[0].numpy().squeeze()
    plt.subplot(4, 5, i + 1)
    plt.imshow(image)
    plt.title(f"True: {true_label} | Pred: {pred}", fontsize=12, color="blue" if pred==true_label else "red")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [24]:
# Interactive prediction function clearly for demo purposes
def predict_single_image(model, image):
    pred = model.predict(tf.expand_dims(image, axis=0))
    pred_label = int(np.round(pred).flatten()[0])

    plt.imshow(image)
    plt.title(f"Model Prediction: {pred_label}", fontsize=14)
    plt.axis('off')
    plt.show()

# Clearly pick a random image from your demo set for interactive demonstration
for img, label in demo_dataset.take(1):
    predict_single_image(custom_mmsen_model, img.numpy()[0])

In [25]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# Interactive prediction function showing both true and predicted labels
def predict_single_image(model, image, true_label):
    pred = model.predict(tf.expand_dims(image, axis=0))
    pred_label = int(np.round(pred).flatten()[0])

    plt.imshow(image)
    plt.title(f"True Label: {true_label} | Model Prediction: {pred_label}",
              fontsize=14,
              color="blue" if pred_label == true_label else "red")
    plt.axis('off')
    plt.show()

# Clearly demonstrate predictions on 4 images interactively
for img_batch, label_batch in demo_dataset.take(4):
    image = img_batch[0].numpy().squeeze()
    true_label = label_batch.numpy()[0]
    predict_single_image(custom_mmsen_model, image, true_label)


In [31]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc, classification_report

def plot_precision_recall_curve(y_true, y_pred_prob):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
    auc_pr = auc(recall, precision)
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.', label=f'PR Curve (AUC = {auc_pr:.4f})')
    plt.title("Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.legend()
    plt.show()

def plot_roc_curve(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    auc_roc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, marker='.', label=f'ROC Curve (AUC = {auc_roc:.4f})')
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.title("ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.show()

def plot_class_accuracy(y_true, y_pred):
    from sklearn.metrics import accuracy_score
    classes = ['Non-Metastatic', 'Metastatic']
    accuracies = [accuracy_score(y_true == i, y_pred == i) for i in range(2)]
    plt.figure(figsize=(8, 6))
    plt.bar(classes, accuracies, color=['blue', 'orange'])
    plt.title("Class-wise Accuracy")
    plt.ylabel("Accuracy")
    plt.show()

def plot_prediction_distribution(y_pred_prob):
    plt.figure(figsize=(8, 6))
    plt.hist(y_pred_prob, bins=30, alpha=0.7, label='Predicted Probabilities')
    plt.title("Prediction Probability Distribution")
    plt.xlabel("Probability")
    plt.ylabel("Frequency")
    plt.show()

def plot_misclassified_samples(model, dataset, num_samples=10):
    misclassified = []
    for images, labels in dataset:
        predictions = model.predict(images)
        preds = np.round(predictions).astype(int).flatten()
        for img, label, pred in zip(images, labels, preds):
            if label.numpy() != pred:
                misclassified.append((img, label.numpy(), pred))
            if len(misclassified) >= num_samples:
                break
        if len(misclassified) >= num_samples:
            break

    plt.figure(figsize=(15, 10))
    for i, (img, true_label, pred_label) in enumerate(misclassified):
        plt.subplot(2, 5, i+1)
        plt.imshow(img.numpy())
        plt.title(f"True: {true_label} | Pred: {pred_label}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Usage after model evaluation
y_true, y_pred_prob = [], []
for images, labels in ds_test:
    preds = custom_mmsen_model.predict(images).flatten()
    y_true.extend(labels.numpy())
    y_pred_prob.extend(preds)

# Plot visuals
plot_precision_recall_curve(y_true, y_pred_prob)
plot_roc_curve(y_true, y_pred_prob)
plot_class_accuracy(np.array(y_true), np.round(y_pred_prob).astype(int))
plot_prediction_distribution(y_pred_prob)
plot_misclassified_samples(custom_mmsen_model, ds_test)


In [36]:
!pip install nbconvert nbformat
!jupyter nbconvert --to notebook --ClearOutputPreprocessor.enabled=True --inplace your_notebook.ipynb